In [5]:
import os
from model import GPTConfig, GPT
import numpy as np
import networkx as nx
import argparse
import pickle
import re
import torch
from stokenizer import STokenizer
import importlib
import model as model_module
importlib.reload(model_module)
from model import GPTConfig, GPT


# def parse_args():
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--ckpt_iter', type=int, default=240000)
#     parser.add_argument('--config', type=str, default='1_4_320')
#     parser.add_argument('--temperature', type=float, default=0.01)
#     parser.add_argument('--device', type=str, default='cuda:0')
#     return parser.parse_args()

# args = parse_args()

# ckpt_iter = args.ckpt_iter
# device = args.device
# temperature = args.temperature
# config = args.config
ckpt_iter =  3000
temperature = 0.5
device = 'cuda:0'
config = '12_12_768_seed33333' #'1_4_320_seed111111'
#/home/v-lberrayana/coconut/models/MDM/out/12_12_768_seed111111/2000_ckpt.pt

meta_path = f'meta.pkl'

print(f"Loading meta from {meta_path}...")
with open(meta_path, 'rb') as f:
    meta = pickle.load(f)

block_size = 200

#meta['block_size']



out_dir = f'out/{config}/'


ckpt_path = os.path.join(out_dir, f'{ckpt_iter}_ckpt.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
gptconf = GPTConfig(**checkpoint['model_args'])

model = GPT(gptconf)
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)

model.eval()
model.to(device)






import json
data_path = "/home/v-lberrayana/coconut/data/theoritical_paper/prosqa_test_graph_4_coconut.json"
with open(data_path, 'r') as f:
    test_data = json.load(f)


from transformers import AutoTokenizer
# Initialize tokenizer
tokenizer = STokenizer()

pad_token_id = tokenizer.vocab[tokenizer.pad_token]
mask_token_id = tokenizer.vocab[tokenizer.mask_token]
vocab_size = tokenizer.vocab_size

print(f"Loaded {len(test_data)} samples for testing")


#print test data
for i in range(3):
    print(f"Sample {i}:")
    print(test_data[i])
    



Loading meta from meta.pkl...


FileNotFoundError: [Errno 2] No such file or directory: 'out/12_12_768_seed33333/3000_ckpt.pt'

In [29]:
# Check what's in the checkpoint's model_args
print("Checkpoint model_args:")
print(checkpoint['model_args'])
print("\nKeys in checkpoint:")
print(checkpoint.keys())

Checkpoint model_args:
{'n_layer': 12, 'n_head': 12, 'n_embd': 768, 'block_size': 670, 'bias': False, 'vocab_size': 50258, 'mask_token_id': 50257, 'dropout': 0.0}

Keys in checkpoint:
dict_keys(['model', 'optimizer', 'model_args', 'iter_num', 'best_val_loss', 'config'])


In [27]:
# # Reload the model module to pick up any changes
# import importlib
# import model as model_module
# importlib.reload(model_module)
# from model import GPTConfig, GPT

# # Now try loading the checkpoint
# gptconf_new = GPTConfig(**checkpoint['model_args'])
# print("✅ Successfully created GPTConfig with mask_token_id!")
# print(f"Config: {gptconf_new}")

✅ Successfully created GPTConfig with mask_token_id!
Config: GPTConfig(block_size=670, vocab_size=50258, mask_token_id=50257, n_layer=12, n_head=12, n_embd=768, dropout=0.0, bias=False)


ALWAYS RUN THIS :

In [ ]:
import random
def process_dataset(sample):
        # Shuffle edges for robustness
    random.shuffle(sample['edges'])
        
        # Construct the question part
    question = "<bos> " + "|".join([f" {e[0]} {e[1]} " for e in sample['edges']]).strip() + " [Q] "
        
        # Randomly order target and neg_target in the question
    if random.random() < 0.5:
        question += f"{sample['target']} {sample['neg_target']}"
    else:
        question += f"{sample['neg_target']} {sample['target']}"
            
    question += f" [R] {sample['root']}"
        
        # Construct the chain of thought (optimal path) and answer
    current_node = sample['root']
    continuation = ""
    for i in range(1, 10):
        if str(i) in sample["neighbor_k"]:
            next_node = random.choice(
                [n for n in sample["neighbor_k"][str(i)] if [current_node, n] in sample['edges']]
            )
            continuation += f" {next_node}"
            current_node = next_node
        
    continuation += f" [A] {sample['target']} <eos>"
    print(continuation)
   
        
        # Tokenize question and continuation
    question_tokenized = tokenizer.encode(question, add_special_tokens=False)
    continuation_tokenized = tokenizer.encode(continuation, add_special_tokens=False)
        
    tokens = question_tokenized + continuation_tokenized
        
    processed_sample = {
        "input_ids": tokens,
        "labels": [-100] * len(question_tokenized) + continuation_tokenized,
        "attention_mask": [1] * len(tokens),
        "position_ids": list(range(len(tokens))),
    }

    print("###########################")
    print(question + continuation)
    print("BLOCK SIZE", len(question_tokenized) + len(continuation_tokenized))           
    return question_tokenized, continuation_tokenized



def is_correct(generated_answer, ground_truth):
    """
    Compare only the last sentence (after the final period) of both answers.
    Returns True if they match.
    """

    print("Generated answer:", generated_answer)
    print("Ground truth answer:", ground_truth)
        
    # Function to extract the last sentence after the last period
    def get_answer(sequence):
        if sequence is None:
            return ""
        if not isinstance(sequence, str):
            sequence = str(sequence)
        matches = re.findall(r'\[A\]\s*(.*)', sequence, flags=re.S)
        ans = matches[-1] if matches else ""
        # collapse all whitespace and strip edges
        return " ".join(ans.split()).strip()
    
    # Extract last sentences
    ground_truth_last = ground_truth
    generated_last = get_answer(generated_answer)

    print(f"DEEEEFFFF : Ground TRUTH: '{ground_truth_last}'")
    print(f"DEEEEFFFF : Generated : '{generated_last}'")

    if ground_truth_last == "" or generated_last == "":
        return False
    try:
        return int(ground_truth_last) == int(generated_last)
    except (ValueError, TypeError):
        return False


In [4]:
import re

results = []
correct_count = 0
total_count = 0

temperature = 0.1 #closer to zero means more deterministic

# We will generate only for a small subset first to monitor memory
max_samples = 500





with torch.inference_mode():
    for sample_idx, sample in enumerate(test_data[1:max_samples]):
        question_ids, answer_ids = process_dataset(sample)
        answer = sample['target']

        # Déterminer combien de tokens masqués ajouter
        # Ajoutons une longueur raisonnable pour la réponse (par exemple 100 tokens)
        num_mask_tokens = max(1, block_size - len(question_ids))
        
        if num_mask_tokens <= 0:
            print(f"[Sample {sample_idx}] Question trop longue, skip")
            continue

        # Créer la séquence: question + tokens masqués
        masked_ids = question_ids + [mask_token_id] * num_mask_tokens
        
        # Tronquer si nécessaire
        if len(masked_ids) > block_size:
            masked_ids = masked_ids[:block_size]

        # Convert to tensor
        input_ids = torch.tensor([masked_ids], dtype=torch.long, device=device)

        # Générer: le modèle va remplacer les tokens masqués
        generated_tensor = model.generate(input_ids, max_new_tokens=500, temperature=temperature, top_k=1 ,pad_token_id=pad_token_id)
        generated_ids = generated_tensor[0].tolist()
            
        answer_start_idx = len(question_ids)
        generated_answer_ids = generated_ids [answer_start_idx:]
            
        # Décoder la réponse générée
        generated_answer = tokenizer.decode(generated_answer_ids, skip_special_tokens=True)
        #remove the pad tokens from the generated answer
        generated_answer = generated_answer.replace(tokenizer.pad_token, '').strip() #IMPORTANT
            
        #print("real answer", answer)
        #print("generated_answer", generated_answer)

        correct = is_correct(generated_answer, answer)
        
        if correct:
            correct_count += 1
        total_count += 1
        print("----")
        print (question_ids )
        print(answer)
        results.append({
            'question': sample['question'],
            'ground_truth': answer,
            'generated': generated_answer,
            'is_correct': correct
        })

        #print(f"question: {sample['question']}")
        #print(f"Reel answer: {answer}")
        #print(f"Generated answer: {generated_answer}")
        print(f"Is Correct: {correct}")
        #print('-' * 80)
        
# Optional: free cached memory after generation
torch.cuda.empty_cache()
print(f"\nCompleted {len(results)} samples")
print(f"Accuracy: {correct_count}/{total_count} = {correct_count/total_count*100:.2f}%" if total_count > 0 else "Accuracy: N/A (no 'is' tokens found)")

# Save results to JSON file
import json
from datetime import datetime

output_filename = f'evaluation_results2_{config}_iter{ckpt_iter}_temp{temperature}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
output_data = {
    'metadata': {
        'config': config,
        'checkpoint_iteration': ckpt_iter,
        'temperature': temperature,
        'total_samples': total_count,
        'correct_count': correct_count,
        'accuracy': (correct_count/total_count*100) if total_count > 0 else 0,
        'timestamp': datetime.now().isoformat()
    },
    'results': results
}

with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"\n✅ Results saved to: {output_filename}")
 

 5 7 14 21 [A] 21 <eos>
###########################
<bos> 14 21 | 7 8 | 12 15 | 1 5 | 5 7 | 1 16 | 4 7 | 6 17 | 9 19 | 19 23 | 11 18 | 2 10 | 4 20 | 0 10 | 1 4 | 6 8 | 0 2 | 12 17 | 8 11 | 11 13 | 5 6 | 2 9 | 2 23 | 1 6 | 7 13 | 7 14 | 1 8 | 5 22 | 3 23 | 10 16 | 4 5 | 8 12 | 13 17 | 9 10 [Q] 23 21 [R] 1 5 7 14 21 [A] 21 <eos>
BLOCK SIZE 114
Generated answer: 4 8 19 23 [A] 23
Ground truth answer: 21
DEEEEFFFF : Ground TRUTH: '21'
DEEEEFFFF : Generated : '23'
----
[32, 14, 21, 34, 7, 8, 34, 12, 15, 34, 1, 5, 34, 5, 7, 34, 1, 16, 34, 4, 7, 34, 6, 17, 34, 9, 19, 34, 19, 23, 34, 11, 18, 34, 2, 10, 34, 4, 20, 34, 0, 10, 34, 1, 4, 34, 6, 8, 34, 0, 2, 34, 12, 17, 34, 8, 11, 34, 11, 13, 34, 5, 6, 34, 2, 9, 34, 2, 23, 34, 1, 6, 34, 7, 13, 34, 7, 14, 34, 1, 8, 34, 5, 22, 34, 3, 23, 34, 10, 16, 34, 4, 5, 34, 8, 12, 34, 13, 17, 34, 9, 10, 35, 23, 21, 36, 1]
21
Is Correct: False
 7 10 12 [A] 12 <eos>
###########################
<bos> 6 11 | 8 13 | 19 20 | 3 10 | 7 10 | 4 13 | 10 17 | 2 19 | 2 5 | 3

## DEBUG

In [ ]:
import re

results = []
correct_count = 0
total_count = 0

temperature = 0.001

# We will generate only for a small subset first to monitor memory
max_samples = 500



def is_correct(generated_answer, ground_truth):
    """
    Compare only the last sentence (after the final period) of both answers.
    Returns True if they match.
    """
        
    # Function to extract the last sentence after the last period
    def get_answer(sequence):
        if sequence is None:
            return ""
        if not isinstance(sequence, str):
            sequence = str(sequence)
        m = re.search(r'\[A\]\s*(.*?)(?:\s*<eos>|$)', sequence, flags=re.S)
        ans = m.group(1) if m else ""
        # collapse all whitespace and strip edges
        return " ".join(ans.split()).strip()
    
    # Extract last sentences
    ground_truth_last = ground_truth #get_answer(ground_truth) 
    generated_last = get_answer(generated_answer)

    print(f"DEEEEFFFF : Ground TRUTH last part: '{ground_truth_last}'")
    print(f"DEEEEFFFF : Generated last part: '{generated_last}'")

    # Compare (case-insensitive and strip whitespace)
    return ground_truth_last.lower() == generated_last.lower()


with torch.inference_mode():
    for sample_idx, sample in enumerate(test_data[1:max_samples]):
        question_ids, answer_ids = process_dataset(sample)
        answer = sample['target']

        # Déterminer combien de tokens masqués ajouter
        # Ajoutons une longueur raisonnable pour la réponse (par exemple 100 tokens)
        num_mask_tokens = max(1, block_size - len(question_ids))
        
        if num_mask_tokens <= 0:
            print(f"[Sample {sample_idx}] Question trop longue, skip")
            continue

        # Créer la séquence: question + tokens masqués
        masked_ids = question_ids + [mask_token_id] * num_mask_tokens
        
        # Tronquer si nécessaire
        if len(masked_ids) > block_size:
            masked_ids = masked_ids[:block_size]

        # Convert to tensor
        input_ids = torch.tensor([masked_ids], dtype=torch.long, device=device)

        # Générer: le modèle va remplacer les tokens masqués
        generated_tensors = model.generate2(input_ids, max_new_tokens=500, temperature=temperature, top_k=1)

        number = 1 
        for generated_tensor in generated_tensors:
            
            generated_ids = generated_tensor[0].tolist()
            
            answer_start_idx = len(question_ids)
            generated_answer_ids = generated_ids #[answer_start_idx:]
            
            # Décoder la réponse générée
            generated_answer = tokenizer.decode(generated_answer_ids, skip_special_tokens=True)
            #remove the pad tokens from the generated answer
            generated_answer = generated_answer.replace(tokenizer.pad_token, '').strip() #IMPORTANT
            print("generated_answer", number, ":", generated_answer)
            number += 1
            
        print("real answer", answer)
        #print("full answer", generated_answer[:answer_start_idx])

        print(xx)
        
        # Décoder la réponse générée
        generated_answer = tokenizer.decode(generated_answer_ids, skip_special_tokens=True)
        #remove the pad tokens from the generated answer
        generated_answer = generated_answer.replace(tokenizer.pad_token, '').strip() #IMPORTANT

        correct = is_correct(generated_answer, answer)
        
        if correct:
            correct_count += 1
        total_count += 1
        
        results.append({
            'question': sample['question'],
            'ground_truth': answer,
            'generated': generated_answer,
            'is_correct': correct
        })

        print(f"question: {sample['question']}")
        print(f"Reel answer: {answer}")
        print(f"Generated answer: {generated_answer}")
        print(f"Is Correct: {correct}")
        print('-' * 80)

        
# Optional: free cached memory after generation
torch.cuda.empty_cache()
print(f"\nCompleted {len(results)} samples")
print(f"Accuracy: {correct_count}/{total_count} = {correct_count/total_count*100:.2f}%" if total_count > 0 else "Accuracy: N/A (no 'is' tokens found)")

# Save results to JSON file
import json
from datetime import datetime

output_filename = f'evaluation_results2_{config}_iter{ckpt_iter}_temp{temperature}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
output_data = {
    'metadata': {
        'config': config,
        'checkpoint_iteration': ckpt_iter,
        'temperature': temperature,
        'total_samples': total_count,
        'correct_count': correct_count,
        'accuracy': (correct_count/total_count*100) if total_count > 0 else 0,
        'timestamp': datetime.now().isoformat()
    },
    'results': results
}

with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"\n✅ Results saved to: {output_filename}")


# Loop

In [ ]:
import os
from model import GPTConfig, GPT
import numpy as np
import networkx as nx
import argparse
import pickle
import re
import torch
from stokenizer import STokenizer
import importlib
import model as model_module
importlib.reload(model_module)
from model import GPTConfig, GPT



device = 'cuda:0'



#temperature = 0.1 #closer to zero means more deterministic

# We will generate only for a small subset first to monitor memory
max_samples = 500

#/home/v-lberrayana/coconut/models/MDM/out/12_12_768_seed111111/2000_ckpt.pt

meta_path = f'meta.pkl'

print(f"Loading meta from {meta_path}...")
with open(meta_path, 'rb') as f:
    meta = pickle.load(f)

block_size = 200

config = '2_8_768_seed444' #'1_4_320_seed111111'
out_dir = f'out/{config}/'




def is_correct(generated_answer, ground_truth):
        """
        Compare only the last sentence (after the final period) of both answers.
        Returns True if they match.
        """

        print("Generated answer:", generated_answer)
        print("Ground truth answer:", ground_truth)
            
        # Function to extract the last sentence after the last period
        def get_answer(sequence):
            if sequence is None:
                return ""
            if not isinstance(sequence, str):
                sequence = str(sequence)
            matches = re.findall(r'\[A\]\s*(.*)', sequence, flags=re.S)
            ans = matches[-1] if matches else ""
            # collapse all whitespace and strip edges
            return " ".join(ans.split()).strip()
        
        # Extract last sentences
        ground_truth_last = ground_truth
        generated_last = get_answer(generated_answer)

        print(f"DEEEEFFFF : Ground TRUTH: '{ground_truth_last}'")
        print(f"DEEEEFFFF : Generated : '{generated_last}'")

        if ground_truth_last == "" or generated_last == "":
            return False
        try:
            return int(ground_truth_last) == int(generated_last)
        except (ValueError, TypeError):
            return False


for temperature in [ 0.5]:
    for i in range(1,20):

        ckpt_iter =  6000 + i*6000
        ckpt_path = os.path.join(out_dir, f'{ckpt_iter}_ckpt.pt')
        checkpoint = torch.load(ckpt_path, map_location=device)
        gptconf = GPTConfig(**checkpoint['model_args'])

        model = GPT(gptconf)
        state_dict = checkpoint['model']
        unwanted_prefix = '_orig_mod.'
        for k,v in list(state_dict.items()):
            if k.startswith(unwanted_prefix):
                state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
        model.load_state_dict(state_dict)

        model.eval()
        model.to(device)



        import json
        data_path = "/home/v-lberrayana/coconut/data/theoritical_paper/prosqa_test_graph_4_coconut.json"
        with open(data_path, 'r') as f:
            test_data = json.load(f)


        from transformers import AutoTokenizer
        # Initialize tokenizer
        tokenizer = STokenizer()

        pad_token_id = tokenizer.vocab[tokenizer.pad_token]
        mask_token_id = tokenizer.vocab[tokenizer.mask_token]
        vocab_size = tokenizer.vocab_size

        print(f"Loaded {len(test_data)} samples for testing")


        #print test data
        for i in range(3):
            print(f"Sample {i}:")
            print(test_data[i])
            
        import re

        results = []
        correct_count = 0
        total_count = 0

        with torch.inference_mode():
            for sample_idx, sample in enumerate(test_data[1:max_samples]):
                question_ids, answer_ids = process_dataset(sample)
                answer = sample['target']

                # Déterminer combien de tokens masqués ajouter
                # Ajoutons une longueur raisonnable pour la réponse (par exemple 100 tokens)
                num_mask_tokens = max(1, block_size - len(question_ids))
                
                if num_mask_tokens <= 0:
                    print(f"[Sample {sample_idx}] Question trop longue, skip")
                    continue

                # Créer la séquence: question + tokens masqués
                masked_ids = question_ids + [mask_token_id] * num_mask_tokens
                
                # Tronquer si nécessaire
                if len(masked_ids) > block_size:
                    masked_ids = masked_ids[:block_size]

                # Convert to tensor
                input_ids = torch.tensor([masked_ids], dtype=torch.long, device=device)

                # Générer: le modèle va remplacer les tokens masqués
                generated_tensor = model.generate(input_ids, max_new_tokens=500, temperature=temperature, top_k=1 ,pad_token_id=pad_token_id)
                generated_ids = generated_tensor[0].tolist()
                    
                answer_start_idx = len(question_ids)
                generated_answer_ids = generated_ids [answer_start_idx:]
                    
                # Décoder la réponse générée
                generated_answer = tokenizer.decode(generated_answer_ids, skip_special_tokens=True)
                #remove the pad tokens from the generated answer
                generated_answer = generated_answer.replace(tokenizer.pad_token, '').strip() #IMPORTANT
                    
                #print("real answer", answer)
                #print("generated_answer", generated_answer)

                correct = is_correct(generated_answer, answer)
                
                if correct:
                    correct_count += 1
                total_count += 1
                print("----")
                print (question_ids )
                print(answer)
                results.append({
                    'question': sample['question'],
                    'ground_truth': answer,
                    'generated': generated_answer,
                    'is_correct': correct
                })

                #print(f"question: {sample['question']}")
                #print(f"Reel answer: {answer}")
                #print(f"Generated answer: {generated_answer}")
                print(f"Is Correct: {correct}")
                #print('-' * 80)
                
        # Optional: free cached memory after generation
        torch.cuda.empty_cache()
        print(f"\nCompleted {len(results)} samples")
        print(f"Accuracy: {correct_count}/{total_count} = {correct_count/total_count*100:.2f}%" if total_count > 0 else "Accuracy: N/A (no 'is' tokens found)")

        # Save results to JSON file
        import json
        from datetime import datetime

        output_filename = f'evaluation_results2_{config}_iter{ckpt_iter}_temp{temperature}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
        output_data = {
            'metadata': {
                'config': config,
                'checkpoint_iteration': ckpt_iter,
                'temperature': temperature,
                'total_samples': total_count,
                'correct_count': correct_count,
                'accuracy': (correct_count/total_count*100) if total_count > 0 else 0,
                'timestamp': datetime.now().isoformat()
            },
            'results': results
        }

        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, indent=2, ensure_ascii=False)

        print(f"\n✅ Results saved to: {output_filename}")
        


/home/v-lberrayana/miniconda3/envs/coconut/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading meta from meta.pkl...
number of parameters: 85.02M
Loaded 419 samples for testing
Sample 0:
{'question': 'Every gerpus is a terpus. Every terpus is a zhorpus. Every lempus is a yerpus. Every boompus is a zhorpus. Every brimpus is a rempus. Every lempus is a jelpus. Every lorpus is a rorpus. Bob is a yerpus. Every worpus is a rempus. Every lempus is a impus. Every rempus is a sterpus. Every yimpus is a zumpus. Every lempus is a yumpus. Every shumpus is a jelpus. Every brimpus is a zhorpus. Every scrompus is a rempus. Every lempus is a wumpus. Sally is a boompus. Sally is a gerpus. Every gerpus is a scrompus. Bob is a wumpus. Every wumpus is a lorpus. Every yerpus is a rorpus. Sally is a terpus. Every gerpus is a zhorpus. Every yimpus is a zhorpus. Every boompus is a terpus. Every gerpus is a worpus. Bob is a lorpus. Every gerpus is a yimpus. Every scrompus is a brimpus. Every lempus is a rorpus. Every lempus is a shumpus. Bob is a jelpus. Sally is a scrompus. Every gerpus is a b

ValueError: invalid literal for int() with base 10: '17 17'